### Introduction

Recommender Systems filter items based on user-item (collaborative filtering ) patterns or item-content (content filtering) patterns. 
Collaborative filtering builds a model from a user's past patterns. It uses items previously purchased or selected and ratings (usually numerical) given by users of these items to make recommendations.
Content filtering on the other hand uses the features of an item to recommend similar items to users.
The focus of this article will be on a variant of collaborative filtering called `Deep Matrix Factorization`


Collaboritive filtering systems require a large amount of data about users in order to make good recommendations. This often leads to 3 problems; Cold Start, Scalability and Sparsity.

1. Cold Start<sup>1</sup> analogous to not having enough data about a user to make good recommendations.
2. Scalabiltiy<sup>2</sup> in respect to the amount of compute needed to build an end-to-end recsys.
3. Sparsity as a reference to the number of items that were successfully rated by users (Usually always really low).


To solve the problem of sparsity, earlier systems relied on filling the missing rating (i.e making the data dense) with the average ratings for each user and item. This was expensive as it increases the amount of data significantly and introduces a lot of noise into the data. Simon Funk <sup>3</sup> while taking part in the Netflix Prize<sup>4</sup> proposed the Matrix Factorization idea for solving this problem. 

### Matrix Factorization

![UserItemRating](/images/user_item.png)

The collection of user, item, rating in a typical RecSys problem can be stored as a `sparse matrix` where users are in `rows` and items are in `columns` with the ratings as values in the matrix. The missing values in the matrix are unobserved instances i.e ratings to predict. and is essentially the task here. The prediction of these ratings  is usually done in one of 3 ways:

1. If the data is numeric and bimodal between two distant values, the mean imputation is used to fill the data with values that otherwise would not exist in the dataset at all.
2. If there are not many missing values then the mean or median value of the observed values is used to fill in the missing values.
3. If the data is categorical, the most frequently observed value is used to fill in the missing values.

The methods above have various challenges ranging from the introduction of noise into the dataset, to overlooking important iteractions between users and items.

Matrix factorization is a solution to this problem that attempts to bypass the challenges above. It assumes that a bunch of hidden feature exists that determine how a user rates an item. For example, Users A and C could give high ratings to a particular Item B if they both like a certain thing about this item. As such, if we can discover what the hidden thing they both like about the item is, we should be able to predict a rating with respect to this user and other items.
Assuming our sparse user-item-rating matrix is denoted by $A$, matrix factorization tries to learn a number of features, say $k$ that represents a user or item. Using the concept of matrix decomposition, $A$ is then decomposed into the dot product of two lower rank matrices $L$ and $U$, such that
        $$A = L.U$$ where
        
          A = Our sparse matrix
          L = user-factor matrix i.e features that stand out about the users 
          U = item-factor matrix i.e features that stand out about the items.                    
The predicted rating for each missing value is then given as the sum of $k$ instances of the $L$ and $U$ reprisented as:
                
$$rating_{i,j} = \sum_{l=1}^k \ L_{i,l} . \ U_{j,l} = U_{i,l}^T.L_{j,l}$$



To get the matrices $L$ and $U$, we need to go back to our matrix $A$.
If $A$ is dense, i.e most of the elements are non-zero, the $L$ is the eigenvectors of $AA^T$ and U, the eigenvectos of $A^TA$. If $A$ is sparse, i.e most of the elements are zero, we can compute $L$ and $U$ using gradient descent. To do this, we set $L$ and $U$ to some initial random values, pick a random instance $B$ and then iteratively find the difference between $B$ and the product of $L$ and $U$. This iteration goes on till we reach a local minimal of the difference. The difference can be calculated using:
    $$d_{i,j}^{2} = \left(d_{i,j} - \overbrace{d_{i,j}}\right)^2$$
    where $d_{i,j}$ = real rating and $\overbrace{d_{i,j}}$ = predicted rating
     
Substituting our initial equation into this, we get:
        $$\left(d_{i,j} - \overbrace{d_{i,j}}\right)^2 = \left(d_{i,j} - \sum_{l=1}^k \ L_{i,l} . \ U_{j,l}\right)^2$$

To reduce $\left(d_{i,j} - \overbrace{d_{i,j}}\right)^2$, i.e the error term, we need to update the values of $L_{i,l}$ and  $U_{j,l}$ at each level. To do this, we differentiate the equation above with respect to both $L_{i,l}$ and  $U_{j,l}$. This gives us:
    $$\frac{\partial d_{i,l}^{2}}{\partial L_{i,l}} = - 2d_{i,j}U_{l,j}$$ and $$\frac{\partial d_{i,l}^{2}}{\partial U_{i,l}} = - 2d_{i,j}L_{i,l} $$
    
At this point, we can ascertain that $L_{i,l}$ can be updated with:
    $$L^{'}_{il} \ =L_{il} + 2\alpha d_{ij} U_{lj} \ $$

Similarly, $U_{i,l}$ can be updated at each point with:
    $$U^{'}_{lj} \ =U_{il} + 2\alpha d_{ij} L_{il} \ $$

As in every intelligent system, there's a need to consider the availability of `biases` in how users rate items. Biases are personal, important and often have a lasting effect in how users behave. Some users can rate items higher or lower in respect to their bias to these items. As a result of this, in making predictions for ratings, it is important we include some form of bias in our equation above. $\overbrace{d_{i,j}}$ above with bias included becomes:
        $$\overbrace{d_{i,j}} = b_{user, i} + b_{item, j} + rating_{i,j}$$ where $b$ is the global bias, $b_{user, i}$ is the bias of a user $i$ and $b_{item, j}$ the bias of an item $j$.

Since the entire matrix factorization idea is an gradient optimizing process, there will be a need to update the biases at each point of descent. We can achieve this by updating our biases at each point with:
    $$b'_{user, i} = b_{user, i} + \alpha \times (d_{ij} - \beta bu_i)$$
       $$b'_{item, i} = b_{item, i} + \alpha \times (d_{ij} - \beta bu_i)$$





To better illustrate matrix factorization, let's try a simple python implementation of the algorithm using a top down approach.

###### Step 1 : Create the user-items-rating matrix

In [177]:


### Simple matrix with ratings as element.
### We're to predict the 0 values which are items that have not been rated yet.

import numpy
user_item_rating = numpy.array([[5, 3, 0, 1],
                                [4, 0, 0, 1],
                                [1, 1, 0, 5],
                                [1, 0, 0, 4],
                                [0, 1, 5, 4]])

###### Step 2 : Find the values of L and U using gradient descent

In [178]:
def find_l_u_gradient_descent(user_item_rating):
    features = 2  # number of hidden factors in our matrix
    alpha = 0.0002  # learning rate i.e $\alpha$
    beta = 0.02     # Bias
    n_epochs = 1000  # number of iterations to run this for
    
    users, items = user_item_rating.shape  ### User and item features
    
    # Randomly initialize users and items
    L = numpy.random.rand(users, features)
    U = numpy.random.rand(items, features)
    U = U.T
    
    samples = [
            (i, j)
            for i in range(users)
            for j in range(items)
            if user_item_rating[i][j] > 0
        ] ### Map of items with no ratings. i.e ratings we're predicting for 
    
    for i in range(n_epochs):
        for i, j in samples:
            d_ij = user_item_rating[i][j] - numpy.dot(L[i,:], U[:,j])     ### Get error term at each point of descent
            for l in range(features):
                L[i][l] = L[i][l] + alpha * (2 * d_ij * U[l][j] - beta * L[i][l]) ### Update L at each feature descent point.
                U[l][j] = U[l][j] + alpha * (2 * d_ij * L[i][l] - beta * U[l][j]) ### Update U at each new feature descent point
    return L, U.T   ### Return L and U

###### Step 3 : Find the dot product of L and U

In [179]:
def find_dot_l_u(l, u):
    return numpy.dot(l, u)

Putting this all together, we get:

In [180]:
L, U = find_l_u_gradient_descent(user_item_rating)

find_dot_l_u(L, U.T)

array([[3.58193084, 2.11016737, 2.5515962 , 2.51643076],
       [2.67882287, 1.53323706, 2.05104122, 2.06722282],
       [2.77042234, 1.34618847, 2.88273825, 3.126065  ],
       [2.03771395, 0.83453415, 2.61521841, 2.94143927],
       [3.41638943, 1.50125544, 4.05995427, 4.51028962]])

A less hacky (and easily more practical) way of computing Matrix Factorization is to use the `SVD` (Singular Value Decomposition) module available in the `surprise` RecSys python package. It should however be noted that SVD is used mostly for dense matrices. It (SVD) makes it easy for the eigenvectors of these matrices to be calculated. The `NMF` ( Non-negative Matrix Factorization) module produces better predictions for sparse matrices since the missing values are included in the algorithm.

An often interesting challenge with RecSys is effectively handling the changing nature of both users and items. This time-drifting nature of data in RecSys, often goes a long way to determining the rating of items, since each new (time-based) selection leads to a change in user taste. Given this, a good RecSys should account for temporal changes in user tastes. Examples of timeseries plots (using the movielens data) showing this changing nature is shown below.

![UserItemRating](/images/ts.png)
*Image Source: [Github](https://github.com/WJMatthew/MovieLens-EDA) License MIT

![UserItemRating](/images/ts_1.png)
*Image Source: [Github](https://github.com/WJMatthew/MovieLens-EDA) License MIT



Items appear to have a very slow change with time. Users on the other hand exhibit frequent, sometimes sudden changes in their interest over time. This reinstates why bias is important in making predictions relating to users. The effect of bias on users can at this point be understood to be more important than that of items. The change is often in two forms.

1. Multiple sources : Both items and users change with time.
2. Multiple targets : Each user/item forms a unique map and change with time.

To account for this change, we can introduce a time based constant, $t$ into our initial equation:
    $$\overbrace{d_{i,j}}(t) = b_{user, i}(t) + b_{item, j}(t) + rating_{i,j}(t)$$

### Deep Matrix Factorization

Since matrix factorization is essentially a linear representation of a matrix (dot product), a new challenge ensues when the matrix contains a lot of non-linear interactions. The success of deep learning in solving complicated machine learning (especially in computer vision and language processing) tasks in mostly non-linear spaces made it a natural solution to solving the non-linearity problem.

There have been a number of implementations of matrix factorization using deep learning.  Restricted Boltzmann Machines [Salakhutdinov
et al., 2007] was firstly proposed to model users’ explicit
ratings on items. Autoencoders and the denoising autoencoders
have also been applied for recommendation [Li et
al., 2015; Sedhain et al., 2015; Strub and Mary, 2015]. The
key idea of these methods is to reconstruct the users’ ratings
through learning hidden structures with the explicit historical
ratings. Implicit feedback is also applied in this research line
of deep learning for recommendation.  [He et
al., 2017] also attempted to model the user-item interactions
with a multi-layer feedforward neural network. Regardless of the implementation chosen, the architecture is pretty similar.

Given the user-item-rating matrix is represented as $A$, we can set up a simple architecture where users and items are represented as high-dimensional vectors of $A$. i.e $A_{*i}$ represents the $i_{th}$ user and $A_{*j}$ represents the $j_{th}$ item. In each layer of our architecture, we can map the input vectors into another vector in a new space configuration. If we denote the input and output vectors as $x$ and $y$ respectively, the hidden layers, say, $l_{i},i$ can be represented as :
            $$l_{i}, i = 1, ..., N - 1 $$ where N is the number of layers.
  This suggests that our first hidden layer will be:
            $$l_{1} = W_{1}x$$
   Finally, our hidden features can be computed as a function of the product of the weights $W_{N}$ and hidden layers $l_{N}$, i.e
   $$h = f(W_{n}l_{n-1} + b_{N})$$ where $h$ is the hidden feature and $b$ bias. 

Following this pattern, our final architecture ends up to containing two multi-layer networks mapped to hidden layers of low-dimensional vector as shown below. A ReLU activation can be applied at each point of the network and the similarity between hidden features calculated using cosine similarity.

![DMF Architecture](/images/dmf_layers.png)

Deep matrix factorization provides a leverage over the normal matrix factorization in that output size of the embedded layer is not restricted to a particular value. This is useful in the case where one dimension may be significantly larger than the other and thus requires training a massive number of features. 

Below is an implementation of the DMF using Keras.

In [214]:
from keras.layers import Embedding, Reshape, Dense, dot
from keras.models import Sequential

class DMF(Sequential):
    def __init__(self, data, features= 2 , alpha= 0.0002, beta = 0.02 , n_epochs= 1000, **kwargs):
        self.features = features  # number of hidden factors in our matrix
        self.alpha = alpha  # learning rate i.e $\alpha$
        self.beta = beta    # Bias
        self.n_epochs = n_epochs  # number of iterations to run this for
        self.l_count = 5
        self.data = data
        self.users, self.items = self.data.shape  ### User and item features
        
        self.setup_L()
        self.setup_U()
        
        super(DMF, self).__init__(**kwargs)
        
        self.add(dot([L, U], axes=1, normalize=False))
        
        
    def setup_L(self):
        L = Sequential()
        for i in range(self.l_count):
            L.add(Dense(data.shape[0], activation='relu'))
        l_embedding = Embedding(users, features, input_length=1)
        L.add(l_embedding)
        L.add(Reshape((features,)))
        return L
    
    def setup_U(self):
        U = Sequential()
        for i in range(self.l_count):
            U.add(Dense(data.shape[0], activation='relu'))
        u_embedding = Embedding(users, features, input_length=1)
        U.add(u_embedding)
        U.add(Reshape((features,)))
        return U
    
    def predict_rating(self, user, item):
        return self.predict([np.array([user]), np.array([item])])[0][0]
    

### Conclusion

There's no general rule as to which of vanilla matrix factorization or deep matrix factorization will perfectly fit a problem. Most times, it doesn't hurt to try out both and compare performance. The most compromising issues with the vanilla implementation is inclusion on non-linear interactions in the matrix while for deep matrix factorization, it can be either negative sampling ratio, depth of network(too deep? overfits . too shallow? underfits) and the factors of the final hidden state. As a general form of reference, [Li et al., 2015; Sedhain et al., 2015; Strub and Mary, 2015] all have performance benchmarks for the implementations.

Thank you for reading.

*This post was written entirely in the IPython notebook.*

## Resources

1. [https://www.yuspify.com/blog/cold-start-problem-recommender-systems/](https://www.yuspify.com/blog/cold-start-problem-recommender-systems/)
2. [https://dspace.mit.edu/handle/1721.1/99785](https://dspace.mit.edu/handle/1721.1/99785)
3. [https://sifter.org/simon/journal/20061211.html](https://sifter.org/simon/journal/20061211.html)
4. [https://www.netflixprize.com/](https://www.netflixprize.com/)
1. [https://arxiv.org/pdf/1809.02131.pdf](https://arxiv.org/pdf/1809.02131.pdf)
2. [https://github.com/maciejkula/triplet_recommendations_keras](https://github.com/maciejkula/triplet_recommendations_keras)
3. [https://github.com/bradleypallen/keras-movielens-cf](https://github.com/bradleypallen/keras-movielens-cf)
4. [https://colab.research.google.com/github/jmschrei/notebooks](https://colab.research.google.com/github/jmschrei/notebooks/blob/master/MXNet%20Deep%20Matrix%20Factorization/MxNet_Deep_Matrix_Factorization.ipynb#scrollTo=KgozQGuYAZP0)
5. [http://blog.richardweiss.org/2016/09/25/movie-embeddings.html](http://blog.richardweiss.org/2016/09/25/movie-embeddings.html)
6. [https://www.ijcai.org/proceedings/2017/0447.pdf](https://www.ijcai.org/proceedings/2017/0447.pdf)
7. [https://github.com/khanhnamle1994/movielens/blob/master/CFModel.py](https://github.com/khanhnamle1994/movielens/blob/master/CFModel.py)
